# A FULLY CUSTOMISABLE NEURAL NETWORK IN PYTHON FROM SCRATCH
Posted on March 30, 2018 by Suyog. _[Visit the article on ML Endeavours](https://mlendeavours.wordpress.com/2018/03/30/a-fully-customizable-neural-network-in-python-from-scratch/)_
![alt text](./neural_network.png "A simple feedforward neural network")


Import numpy.

In [2]:
import numpy as np
np.random.seed(777)

Construct the class.

In [3]:
class NN(object):
    """
    A network that uses sigmoid activation function.
    """
    
    
    def __init__(self):
 
        self.nodes = []
        self.layers = {}
        self.weights = {}
        self.n_classes = 0        

The <kbd>add_layer</kbd> function.

In [4]:
def add_layer(self, n_nodes, output_layer=False):
    """
    Adds a layer of specified no. of output nodes.
    For the output layer, the flag  output_layer must be True.
    A network must have an output layer.
    """

    if not output_layer:
        self.nodes.append(n_nodes)
    else:
        self.n_classes = n_nodes

The <kbd>sigmoid</kbd> function.

In [5]:
def sigmoid(self, z):
    """
    Calculates the sigmoid activation function.
    """
 
    return 1 / (1 + np.exp(-z))

The <kbd>predict</kbd> function.

In [6]:
def predict(self, x, to_predict=True, argmax=True, rand_weights=False):
    """
    Performs a pass of forward propagation.
    If to_predict is set to True, trained weights are used
    and predictions are returned in a single vector
    with labels from 0 to (n_classes - 1).
    """
 
    nodes = self.nodes
    layers = {}
    weights = {}
 
    # -------------- for input layer
    m = x.shape[0]
    x = np.append(np.ones(m).reshape(m, 1), x, axis=1)
    layers['a%d' % 1] = x
 
    # --------------- for dense layers
    for i in range(len(nodes)):
        m, n = x.shape
        w = np.random.randn(nodes[i], n) if rand_weights else self.weights['w%d' % (i + 1)]
        z = x.dot(w.T)
        a = np.append(np.ones(m).reshape(m, 1), self.sigmoid(z), axis=1)
 
        if not to_predict:
            layers['a%d' % (i + 2)] = a # We start from a2, as a1 is already done.
            weights['w%d' % (i + 1)] = w # We start from w1
        x = a # to repeat the same procedure for the next layer.
 
    # --------------- for output layer
    m, n = x.shape
    w = np.random.randn(self.n_classes, n) if rand_weights else self.weights['w%d' % (len(nodes) + 1)]
    z = x.dot(w.T)
    a = self.sigmoid(z)
    output = a
 
    if not to_predict:
        layers['a%d' % (len(layers) + 1)] = a
        weights['w%d' % (len(weights) + 1)] = w
 
    if to_predict:
        return np.argmax(output, axis=1)
    elif rand_weights:
        self.layers = layers
        self.weights = weights
    else:
        return layers, weights

The <kbd>cost</kbd> function.

In [7]:
def cost(self, x, y, lamda=0):
    """
    Calculates the cost for given data and labels, with trained weights.
    """
 
    weights = self.weights
    layers, _ = self.predict(x, predict=False)
 
    m, n = x.shape
    reg2 = 0 # The regularization term
    for i in range(len(weights)):
        reg2 += np.sum(weights['w%d' % (i + 1)][:, 1:] ** 2) # L2 regularization
 
    j = (-1 / m) * np.sum(y.T.dot(np.log(layers['a%d' % (len(layers))])) +
                          (1 - y).T.dot(np.log(1 - layers['a%d' % (len(layers))]))) + (lamda / (2 * m)) * reg2
 
    return j

The <kbd>fit</kbd> function.

In [8]:
def fit(self, data, labels, test=[], test_labels=[], alpha=0.01, lamda=0, epochs=50):
    """
    Performs specified no. of epoches.
    One epoch = one pass of forward propagation + one pass of backpropagation.
    """
    
    self.predict(data, predict=False, rand_weights=True)
    
    for epoch in range(epochs):
        layers, weights = self.predict(data, predict=False, rand_weights=False)
        m, n = data.shape # Needed later in code
        
        # ----------------- Calculating del terms
        delta_n = layers['a%d' % (len(layers))] - labels
        delta_n_1 = delta_n.dot(weights['w%d' % (len(weights))]) * layers['a%d' % (len(layers) - 1)] * \
                            (1 - layers['a%d' % (len(layers) - 1)])

        dels = {
                'del%d' % (len(layers)): delta_n,
                'del%d' % (len(layers) - 1): delta_n_1
                }
        
        for i in range(len(weights) - 2):
                    delta = delta[:, 1:].dot(weights['w%d' % (len(layers) - 2 - i)]) * \
                            layers['a%d' % (len(layers) - 2 - i)] * (1 - layers['a%d' % (len(layers) - 2 - i)])
                    dels['del%d' % (len(layers) - 2 - i)] = delta  
                    
        # ------------------ Calculating grad and regularization terms
        grads = {
                'grad%d' % (len(weights)): (1 / m) * (
                    dels['del%d' % (len(layers))].T.dot(layers['a%d' % (len(weights))]))
            }

        regs = {
                'reg%d' % (len(weights)): (lamda / m) * weights['w%d' % (len(weights))]
            }

        for i in range(len(weights) - 1):
                grad = (1 / m) * \
                       (dels['del%d' % (len(layers) - 1 - i)][:, 1:].T.dot(layers['a%d' % (len(weights) - 1 - i)]))

                grads['grad%d' % (len(weights) - 1 - i)] = grad

                reg = (lamda / m) * weights['w%d' % (len(weights) - 1 - i)]
                reg[:, 0] = 0
                regs['reg%d' % (len(weights) - 1 - i)] = reg

        # ----------------- Updating Parameters
        for i in range(1, len(weights) + 1):
            weights['w%d' % i] = weights['w%d' % i] - alpha * grads['grad%d' % i] - regs['reg%d' % i]

        self.layers = layers
        self.weights = weights

        # ------------------ Progress bar
        print("\r" + "{}% |".format(int(100 * i / epochs) + 1) + '#' * int((int(100 * i / epochs) + 1) / 5) +
              ' ' * (20 - int((int(100 * i / epochs) + 1) / 5)) + '|',
              end="") if not i % (epochs / 100) else print("", end="")

        # ------------------ Train accuracy and cost
        acc = 100 * np.sum(np.argmax(layers['a%d' % (len(layers))], axis=1) == np.argmax(labels, axis=1)) / m
        j_train = float(self.cost(data, labels, lamda=lamda))

        # ------------------ Test accuracy and cost (if valid)
        if len(test):
            m1, n1 = test.shape
            j_test = float(self.cost(test, test_labels, lamda=lamda))
            test_prediction = self.predict(test)
            acc_test = 100 * np.sum(test_prediction == np.argmax(test_labels, axis=1)) / m1
            print('Train cost: %0.2f\tTrain acc.: %0.2f%%\tTest cost: %0.2f\tTest acc.: %0.2f%%' % (j, acc, j_test, acc_test))
        else:
            print('cost: %0.2f\tacc.: %0.2f%%' % (j, acc))
    
    # ------------ Final cost and accuracy
    if len(test):
        print('Final train cost: %0.2f\tFinal train acc.: %0.2f%%\tFinal test cost: %0.2f\tFinal test acc.: %0.2f%%' % (j_train, acc_train, j_test, acc_test))
    else:
        print('Final cost: %0.2f\tFinal acc.: %0.2f%%' % (j_train, j_acc))
        